In [5]:
from qazure import get_llm, get_embedder
import logging
import sys

import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings

In [6]:
logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
llm = get_llm()
embed_model = get_embedder()

In [12]:
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
# load documents
documents = SimpleDirectoryReader(input_files=["paul_graham_essay.txt"]).load_data()

In [10]:
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"


In [13]:
vector_store = QdrantVectorStore(client=client, collection_name="paul_graham")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/paul_graham/exists "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/paul_graham/exists "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:6333/collections/paul_graham/exists "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/

In [14]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azu

In [15]:
display(Markdown(f"<b>{response}</b>"))

<b>Growing up, the author engaged in writing and programming outside of school. He wrote short stories, which he described as lacking plot but featuring characters with strong feelings. In programming, he started with an IBM 1401 computer in junior high school, experimenting with early versions of Fortran, though he found it limiting. Later, with the advent of microcomputers, he became more involved, programming on a TRS-80 and creating simple games, a model rocket prediction program, and a word processor. These activities marked his early interests and efforts in creative and technical pursuits.</b>

In [16]:
loaded_index = VectorStoreIndex.from_vector_store(
    vector_store,
    # Embedding model should match the original embedding model
    # embed_model=Settings.embed_model
)

In [17]:
# set Logging to DEBUG for more detailed outputs
query_engine = loaded_index.as_query_engine()
response = query_engine.query(
    "What did the author do after his time at Viaweb?"
)

INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:6333/collections/paul_graham/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://evaln-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-05-01-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://evaln-openai.openai.azu

In [19]:
response

Response(response='After his time at Viaweb, the author transitioned to working at Yahoo following the acquisition of his company. However, he eventually left Yahoo in the summer of 1999 after his stock options vested. He decided to pursue painting, which had been his original goal for getting rich. He initially started painting in California but struggled with energy and ambition, partly due to isolation. In desperation, he returned to New York, resumed his old lifestyle, and began painting more effectively while experimenting with new techniques. He also started looking for an apartment to buy and reflected on his experiences during this period.', source_nodes=[NodeWithScore(node=TextNode(id_='12f6b68c-899f-4dcf-bbf7-9a93533db7ed', embedding=None, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2025-05-05', 'last_modified_date': '2025-05-05'}, excluded_embed_metadata_keys=['file_nam

In [20]:
import llama_index.core as core
dir(core)

['BaseCallbackHandler',
 'BasePromptTemplate',
 'Callable',
 'ChatPromptTemplate',
 'ComposableGraph',
 'Document',
 'DocumentSummaryIndex',
 'GPTDocumentSummaryIndex',
 'GPTKeywordTableIndex',
 'GPTListIndex',
 'GPTRAKEKeywordTableIndex',
 'GPTSimpleKeywordTableIndex',
 'GPTTreeIndex',
 'GPTVectorStoreIndex',
 'IndexStructType',
 'KeywordTableIndex',
 'KnowledgeGraphIndex',
 'ListIndex',
 'MockEmbedding',
 'NullHandler',
 'Optional',
 'Prompt',
 'PromptHelper',
 'PromptTemplate',
 'PropertyGraphIndex',
 'QueryBundle',
 'RAKEKeywordTableIndex',
 'Response',
 'SQLContextBuilder',
 'SQLDatabase',
 'SQLDocumentContextBuilder',
 'SelectorPromptTemplate',
 'ServiceContext',
 'Settings',
 'SimpleDirectoryReader',
 'SimpleKeywordTableIndex',
 'StorageContext',
 'SummaryIndex',
 'TreeIndex',
 'VectorStoreIndex',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'async_u